In [1]:
DATA_TRAIN_PROCESSED = '../data/prepocessed/train.csv'
LOG_RESULTS = '../models/log/metrics_results.csv'
REPORT = '../reports/results/'

# Import

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore")

C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
from statsmodels.tsa.forecasting.theta import ThetaModel
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Define

## Model

In [114]:
from statsmodels.tsa.forecasting.theta import ThetaModel

class THETAModel:
    def __init__(self, train_data):
        """
        Khởi tạo mô hình Theta.
        
        Args:
            train_data (pd.Series): Dữ liệu huấn luyện (chuỗi thời gian).
        """
        self.train_data = train_data
        self.model = None

    def fit(self, periods=0):
        """
        Huấn luyện mô hình Theta với chu kỳ mùa vụ được chỉ định.

        Args:
            periods (int): Độ dài của chu kỳ mùa vụ.
        """
        if not isinstance(periods, int) or periods <= 0:
            raise ValueError("periods phải là số nguyên dương.")

        self.model = ThetaModel(self.train_data, period=periods).fit()
        print(f"Theta Model trained successfully with period={periods}.")

    def predict(self, length):
        """
        Dự báo nhiều bước phía trước.

        Args:
            length (int): Số bước cần dự báo.

        Returns:
            np.array: Giá trị dự báo.
        """
        if self.model is None:
            raise ValueError("Model chưa được huấn luyện. Vui lòng gọi fit() trước.")

        forecast = self.model.forecast(steps=length)
        return forecast.to_numpy(), None

## Task

In [115]:
def normalize_data(data, method="minmax"):
    """Chuẩn hóa dữ liệu theo phương pháp MinMax hoặc StandardScaler."""
    if method == "minmax":
        scaler = MinMaxScaler(feature_range=(0, 1))
    elif method == "standard":
        scaler = StandardScaler()
    else:
        return data, None

    data_scaled = scaler.fit_transform(np.array(data).reshape(-1, 1))
    return data_scaled, scaler

In [116]:
def inverse_scale(data_scaled, scaler):
    """
    Đảo ngược quá trình chuẩn hóa dữ liệu.

    Parameters:
        data_scaled (np.array): Dữ liệu đã được chuẩn hóa.
        scaler: Đối tượng scaler đã sử dụng để chuẩn hóa dữ liệu.

    Returns:
        np.array: Dữ liệu gốc sau khi đảo ngược chuẩn hóa.
    """
    if scaler is None:
        return data_scaled  
    data_original = scaler.inverse_transform(np.array(data_scaled).reshape(-1, 1))
    return data_original.flatten()

In [117]:
def plot_forecast(train_data, y_test, y_pred, model, index, conf_int=None):
    """
    Vẽ và lưu biểu đồ dự báo.

    Args:
        train_data (pd.Series): Dữ liệu huấn luyện.
        y_test (pd.Series): Dữ liệu thực tế.
        y_pred (np.array): Giá trị dự báo.
        model: Đối tượng model đã được huấn luyện (có thuộc tính `__class__.__name__`).
        index (int): Index của dòng metrics đã lưu.
        conf_int (np.array, optional): Khoảng tin cậy của dự báo.
    """
    plt.figure(figsize=(12, 6))

    train_index = train_data.index
    test_index = y_test.index

    model_name = model.__class__.__name__

    plt.plot(train_index, train_data, label="Original Data", color='gray', alpha=0.5)
    plt.plot(test_index, y_test, label="Actual", color='blue')
    plt.plot(test_index[:len(y_pred)], y_pred, label="Predicted", color='red', linestyle='dashed')

    # Vẽ khoảng tin cậy nếu có
    if conf_int is not None:
        conf_int = np.array(conf_int).reshape(-1, 2)
        lower_bound, upper_bound = conf_int[:, 0], conf_int[:, 1]
        plt.fill_between(test_index[:len(y_pred)], lower_bound, upper_bound, 
                         color='pink', alpha=0.3, label="Confidence Interval")

    # Cài đặt đồ thị
    plt.title(f"{model_name} Forecast")
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.legend(ncol=4)

    # Lưu hình ảnh với tên index_{Model}.png
    file_path = f"{REPORT}_{index}_{model_name}.png"
    plt.savefig(file_path, dpi=300)
    plt.close()

    print(f"Plot saved to {file_path}")


In [118]:
from sklearn.metrics import mean_absolute_percentage_error, r2_score
from datetime import datetime

def compute_metrics(model, scaler, y_test, y_pred):
    """
    Tính toán các chỉ số đánh giá mô hình và trả về dưới dạng một dòng DataFrame.

    Args:
        model (object): Đối tượng mô hình (VD: ARIMA, RandomForest, LSTM,...)
        scaler (object): Bộ scaler đã sử dụng (VD: MinMaxScaler, StandardScaler,...)
        y_test (array-like): Giá trị thực tế
        y_pred (array-like): Giá trị dự đoán

    Returns:
        pd.DataFrame: Một dòng DataFrame chứa metrics đánh giá mô hình
    """

    model_name = model.__class__.__name__

    try:
        model_params = model.get_params()
    except AttributeError:
        try:
            model_params = model.order  # Cho ARIMA
        except AttributeError:
            model_params = "Unknown"

    scaler_name = scaler.__class__.__name__
    execution_date = datetime.today().strftime("%d-%m-%Y")
    mape = mean_absolute_percentage_error(y_test, y_pred)
    rmse = np.sqrt(np.mean((y_test - y_pred) ** 2))
    r2 = r2_score(y_test, y_pred)

    result = pd.DataFrame([{
        "Model": model_name,
        "Scaler": scaler_name,
        "Params": str(model_params),
        "MAPE": mape,
        "RMSE": rmse,
        "R2 Score": r2,
        "Execution Date": execution_date
    }])

    return result


In [119]:
import os

def save_metrics_to_csv(df, file_path):
    """
    Lưu DataFrame metrics vào file CSV. Nếu file chưa tồn tại thì tạo mới, 
    nếu đã tồn tại thì append dữ liệu vào.

    Args:
        df (pd.DataFrame): DataFrame chứa metrics của mô hình.
        file_path (str): Đường dẫn đến file CSV cần lưu.

    Returns:
        int: Index của dòng vừa được lưu.
    """
    file_exists = os.path.isfile(file_path)

    if file_exists:
        existing_df = pd.read_csv(file_path)
        last_index = existing_df.index[-1] + 1 if not existing_df.empty else 0
    else:
        last_index = 0

    df.index = range(last_index, last_index + len(df))

    # Lưu vào CSV
    df.to_csv(file_path, mode="a", index=False, header=not file_exists, encoding="utf-8")
    
    print(f"Metrics saved to {file_path}")
    
    return last_index 


# Run

In [120]:
data = pd.read_csv(DATA_TRAIN_PROCESSED)

* Configs

In [121]:
normalize = None

* Process

In [122]:
data_copy = data.copy()
data_copy["Date"] = pd.to_datetime(data_copy["Date"])
data_copy.set_index("Date", inplace=True)
data_copy = data_copy.asfreq("D")

* Split

In [123]:
split_idx = int(len(data_copy) * 0.8)
train_data = data_copy.iloc[:split_idx]
val_data = data_copy.iloc[split_idx:]
length_pred = len(val_data)

* Scale

In [124]:
train_scaled, scaler = normalize_data(train_data, method=normalize)

* Init model

In [125]:
model = THETAModel(train_scaled)

* fit model

In [127]:
model.fit(365)

Theta Model trained successfully with period=365.


* forecast

In [143]:
data_copy[data_copy.isna().any(axis=1)]

,temperatures
Date,
1984-12-31,NaN


In [128]:
predictions, conf_int = model.predict(length_pred)

* Inverse scale

In [108]:
if scaler is not None:
    y_pred = inverse_scale(predictions, scaler)
    conf_int[:,0] = inverse_scale(conf_int[:,0], scaler)
    conf_int[:,1] = inverse_scale(conf_int[:,1], scaler)
else:
    y_pred = predictions

* Evaluate

In [109]:
df_metrics = compute_metrics(model, scaler, val_data.values, y_pred)
df_metrics

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

* Save results

In [522]:
# Save metrics
index = save_metrics_to_csv(df_metrics, LOG_RESULTS)
# Save plot
plot_forecast(train_data, val_data, y_pred, model, index=index, conf_int=conf_int)
pd.read_csv(LOG_RESULTS)

Metrics saved to ../models/log/metrics_results.csv
Plot saved to ../reports/results/_5_ARIMAModel.png


,Model,Scaler,Params,MAPE,RMSE,R2 Score,Execution Date
0,ARIMAModel,StandardScaler,"(0, 1, 3)",0.338293,3.982606,-0.055332,15-02-2025
1,SARIMAModel,StandardScaler,"(3, 0, 1)",0.346660,3.890336,0.035631,15-02-2025
2,ARIMAModel,MinMaxScaler,"(0, 1, 3)",0.338292,3.982621,-0.055340,15-02-2025
3,ARIMAModel,MinMaxScaler,"(1, 0, 3)",0.350286,3.891061,0.046005,15-02-2025
4,ARIMAModel,StandardScaler,"(3, 0, 1)",0.346660,3.890336,0.035631,15-02-2025
5,ARIMAModel,NoneType,"(1, 0, 3)",0.346660,3.890336,0.035631,15-02-2025
